In [ ]:
# Uncomment following lines if you are running on Google Colab
# Drive and folder directory may need to be changed
# from google.colab import drive
# drive.mount('/content/drive')
# %ls /content/drive/MyDrive
# %cd /content/drive/MyDrive/10-701-Project
# %pip install tensorboardX
from train import *
import helpers
import copy, glob
from plots import *

#Plotting
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
FIGDIR = 'fig/'

## Running a specific experiment configuration 

In [ ]:
params = {
    'lr': 0.05,
    'dp': True,
    'clipping': 'Linear',
    'num_microbatches': 32,
    'batch_size': 32,
    'S': 1,
    'z': 1.1,
    'gamma': 0.5,
    'lr_c': 0.2,
    'momentum': 0.5,
    'decay': 0,
    'n_epochs' : 20,
}

run_exp('run_a', params, use_devset=True)

## Fetch experiment results over all runs and plot

### Experiment 1 

In [ ]:
exp, params = helpers.load_exp('central_baseline_fixed')
exp['epoch'] = exp['epoch'] + 1
plot_centralized_fixed_cuttoff_experiment(exp)

### Experiment 2

In [ ]:
exp, params= helpers.load_exp('central_baseline_adaptive_cuttoff')
plot_centralized_adaptive_cuttoff_experiment(exp)